## Comparaison de différents modèles supervisés

### A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.

* Déterminer le modèle le plus performant parmi ceux testés.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


In [8]:
X_OHE = pd.read_csv("X_OHE.csv")
y = pd.read_csv("y.csv")

### --- Modèle 1 : DummyRegressor ---


Copies dédiées

In [9]:
X_dummy = X_OHE.copy()
y_dummy = y.copy()

Séparation apprentissage test

In [10]:
Xtr, Xte, ytr, yte = train_test_split(X_dummy, y_dummy, test_size=0.2, random_state=42)

Validation croisée sur le jeu d’apprentissage

In [11]:
scoring = {"r2":"r2", "mae":"neg_mean_absolute_error", "rmse":"neg_root_mean_squared_error"}
cv_res = cross_validate(DummyRegressor(strategy="median"), Xtr, ytr, cv=5, scoring=scoring, return_train_score=False)

cv_r2   = cv_res["test_r2"].mean()
cv_mae  = -cv_res["test_mae"].mean()
cv_rmse = -cv_res["test_rmse"].mean()

Entraînement sur tout le train

In [12]:
dum = DummyRegressor(strategy="median")
dum.fit(Xtr, ytr)

DummyRegressor(strategy='median')

Inférence

In [13]:
yp_tr = dum.predict(Xtr)
yp_te = dum.predict(Xte)

Métriques apprentissage

In [14]:
r2_tr   = r2_score(ytr, yp_tr)
mae_tr  = mean_absolute_error(ytr, yp_tr)
rmse_tr = root_mean_squared_error(ytr, yp_tr)

Métriques test

In [15]:
r2_te   = r2_score(yte, yp_te)
mae_te  = mean_absolute_error(yte, yp_te)
rmse_te = root_mean_squared_error(yte, yp_te)

In [16]:

print(f"Dummy, CV R2 moyen {cv_r2:.3f}, CV MAE moyen {cv_mae:.2f}, CV RMSE moyen {cv_rmse:.2f}")
print(f"Dummy, Train  R2 {r2_tr:.3f}, MAE {mae_tr:.2f}, RMSE {rmse_tr:.2f}")
print(f"Dummy, Test   R2 {r2_te:.3f}, MAE {mae_te:.2f}, RMSE {rmse_te:.2f}")


Dummy, CV R2 moyen -0.033, CV MAE moyen 22.16, CV RMSE moyen 28.87
Dummy, Train  R2 -0.029, MAE 22.14, RMSE 28.91
Dummy, Test   R2 -0.015, MAE 24.24, RMSE 31.10


### --- Modèle 2 : Régression linéaire avec scaling ---



Copies dédiées


In [17]:
X_lin = X_OHE.copy()
y_lin = y.copy()



Séparation apprentissage et test


In [18]:
Xtr, Xte, ytr, yte = train_test_split(X_lin, y_lin, test_size=0.2, random_state=42)



Estimateur enveloppe, scaling puis modèle


In [19]:
class ScaleRegressor:
    def __init__(self, model):
        self.model = model
        self.scaler = StandardScaler()
        self.cols_ = None
    def fit(self, X_fit, y_fit):
        self.cols_ = X_fit.columns
        Xs = self.scaler.fit_transform(X_fit)
        self.model.fit(Xs, y_fit)
        return self
    def predict(self, X_new):
        Xs = self.scaler.transform(X_new[self.cols_])
        return self.model.predict(Xs)
    def get_params(self, deep=True):
        return {"model": self.model}
    def set_params(self, **params):
        for k,v in params.items(): setattr(self, k, v)
        return self

est = ScaleRegressor(LinearRegression())



Validation croisée sur le jeu d’apprentissage


In [20]:
scoring = {"r2":"r2", "mae":"neg_mean_absolute_error", "rmse":"neg_root_mean_squared_error"}
cv_res = cross_validate(est, Xtr, ytr, cv=5, scoring=scoring, return_train_score=False)

cv_r2   = cv_res["test_r2"].mean()
cv_mae  = -cv_res["test_mae"].mean()
cv_rmse = -cv_res["test_rmse"].mean()



Entraînement complet


In [21]:
est.fit(Xtr, ytr)


Inférence


In [22]:
yp_tr = est.predict(Xtr)
yp_te = est.predict(Xte)


Métriques


In [23]:
r2_tr   = r2_score(ytr, yp_tr);  mae_tr  = mean_absolute_error(ytr, yp_tr);  rmse_tr = root_mean_squared_error(ytr, yp_tr)
r2_te   = r2_score(yte, yp_te);  mae_te  = mean_absolute_error(yte, yp_te);  rmse_te = root_mean_squared_error(yte, yp_te)

In [24]:

print(f"LIN, CV R2 moyen {cv_r2:.3f}, CV MAE moyen {cv_mae:.2f}, CV RMSE moyen {cv_rmse:.2f}")
print(f"LIN, Train  R2 {r2_tr:.3f}, MAE {mae_tr:.2f}, RMSE {rmse_tr:.2f}")
print(f"LIN, Test   R2 {r2_te:.3f}, MAE {mae_te:.2f}, RMSE {rmse_te:.2f}")


LIN, CV R2 moyen -67674643390580577659781120.000, CV MAE moyen 29164045139068.56, CV RMSE moyen 188804055277854.56
LIN, Train  R2 0.463, MAE 15.91, RMSE 20.88
LIN, Test   R2 -63373383382340001205321728.000, MAE 31018374387405.23, RMSE 245811188633766.50


### --- Modèle 3 : SVR avec scaling ---


Copies dédiées

In [25]:
X_svr = X_OHE.copy()
y_svr = y.copy()

Séparation apprentissage et test

In [26]:
Xtr, Xte, ytr, yte = train_test_split(X_svr, y_svr, test_size=0.2, random_state=42)

Estimateur: SVR dans ton wrapper de scaling

In [27]:
est = ScaleRegressor(SVR(kernel="rbf", C=10.0, epsilon=0.1, gamma="scale"))

Validation croisée sur le jeu d’apprentissage

In [28]:
scoring = {"r2":"r2", "mae":"neg_mean_absolute_error", "rmse":"neg_root_mean_squared_error"}
cv_res = cross_validate(est, Xtr, ytr, cv=5, scoring=scoring, return_train_score=False)

cv_r2   = cv_res["test_r2"].mean()
cv_mae  = -cv_res["test_mae"].mean()
cv_rmse = -cv_res["test_rmse"].mean()

c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Entraînement complet

In [29]:
est.fit(Xtr, ytr)

c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Inférence

In [30]:
yp_tr = est.predict(Xtr)
yp_te = est.predict(Xte)

Métriques

In [31]:
r2_tr   = r2_score(ytr, yp_tr);  mae_tr  = mean_absolute_error(ytr, yp_tr);  rmse_tr = root_mean_squared_error(ytr, yp_tr)
r2_te   = r2_score(yte, yp_te);  mae_te  = mean_absolute_error(yte, yp_te);  rmse_te = root_mean_squared_error(yte, yp_te)

In [32]:
print(f"SVR, CV R2 moyen {cv_r2:.3f}, CV MAE moyen {cv_mae:.2f}, CV RMSE moyen {cv_rmse:.2f}")
print(f"SVR, Train  R2 {r2_tr:.3f}, MAE {mae_tr:.2f}, RMSE {rmse_tr:.2f}")
print(f"SVR, Test   R2 {r2_te:.3f}, MAE {mae_te:.2f}, RMSE {rmse_te:.2f}")

SVR, CV R2 moyen 0.179, CV MAE moyen 19.30, CV RMSE moyen 25.74
SVR, Train  R2 0.375, MAE 15.59, RMSE 22.53
SVR, Test   R2 0.213, MAE 20.70, RMSE 27.39


### --- Modèle 4 : RandomForestRegressor ---


Copies dédiées

In [33]:
X_rf = X_OHE.copy()
y_rf = y.copy()


Séparation apprentissage et test

In [34]:
Xtr, Xte, ytr, yte = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)



Estimateur Forêt Aléatoire


In [35]:
est = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

Validation croisée sur le jeu d’apprentissage

In [36]:
scoring = {"r2":"r2", "mae":"neg_mean_absolute_error", "rmse":"neg_root_mean_squared_error"}
cv_res = cross_validate(est, Xtr, ytr, cv=5, scoring=scoring, return_train_score=False)

cv_r2   = cv_res["test_r2"].mean()
cv_mae  = -cv_res["test_mae"].mean()
cv_rmse = -cv_res["test_rmse"].mean()


c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exa


Entraînement complet


In [37]:
est.fit(Xtr, ytr)


c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=400, n_jobs=-1, random_state=42)


Inférence


In [38]:
yp_tr = est.predict(Xtr)
yp_te = est.predict(Xte)


Métriques


In [39]:
r2_tr   = r2_score(ytr, yp_tr);  mae_tr  = mean_absolute_error(ytr, yp_tr);  rmse_tr = root_mean_squared_error(ytr, yp_tr)
r2_te   = r2_score(yte, yp_te);  mae_te  = mean_absolute_error(yte, yp_te);  rmse_te = root_mean_squared_error(yte, yp_te)

In [40]:
print(f"RF, CV R2 moyen {cv_r2:.3f}, CV MAE moyen {cv_mae:.2f}, CV RMSE moyen {cv_rmse:.2f}")
print(f"RF, Train  R2 {r2_tr:.3f}, MAE {mae_tr:.2f}, RMSE {rmse_tr:.2f}")
print(f"RF, Test   R2 {r2_te:.3f}, MAE {mae_te:.2f}, RMSE {rmse_te:.2f}")

RF, CV R2 moyen 0.264, CV MAE moyen 18.58, CV RMSE moyen 24.32
RF, Train  R2 0.904, MAE 6.69, RMSE 8.82
RF, Test   R2 0.343, MAE 18.92, RMSE 25.02


### Optimisation et interprétation du modèle

#### A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn.

### --- RandomForest ---


In [41]:

# Copies dédiées
X_rf = X_OHE.copy()
y_rf = y.copy()


In [42]:

# Split
Xtr, Xte, ytr, yte = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)


In [43]:

# Définition du modèle et de la grille
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1, max_samples = 0.7)

param_grid = {
    "n_estimators": [300, 500],
    "max_depth": [12, 16, None],
    "min_samples_leaf": [2, 5, 10],
    "min_samples_split": [2, 10, 20],
    "max_features": ["sqrt", 0.5]
}
gs = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

gs.fit(Xtr, ytr)
best_rf = gs.best_estimator_
print("Meilleurs hyperparamètres:", gs.best_params_)


c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Meilleurs hyperparamètres: {'max_depth': 16, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}


In [44]:

# Validation croisée du meilleur modèle pour R2, MAE, RMSE
scoring = {"r2": "r2", "mae": "neg_mean_absolute_error", "rmse": "neg_root_mean_squared_error"}
cv_res = cross_validate(best_rf, Xtr, ytr, cv=5, scoring=scoring, return_train_score=False)
cv_r2   = cv_res["test_r2"].mean()
cv_mae  = -cv_res["test_mae"].mean()
cv_rmse = -cv_res["test_rmse"].mean()
zip

c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exa

zip

In [45]:

# Entraînement complet puis métriques cohérentes
best_rf.fit(Xtr, ytr)
yp_tr = best_rf.predict(Xtr)
yp_te = best_rf.predict(Xte)

r2_tr   = r2_score(ytr, yp_tr);  mae_tr  = mean_absolute_error(ytr, yp_tr);  rmse_tr = root_mean_squared_error(ytr, yp_tr)
r2_te   = r2_score(yte, yp_te);  mae_te  = mean_absolute_error(yte, yp_te);  rmse_te = root_mean_squared_error(yte, yp_te)


c:\Users\ethan\Anconda4\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [46]:

print(f"RF-GS, CV R2 moyen {cv_r2:.3f}, CV MAE moyen {cv_mae:.2f}, CV RMSE moyen {cv_rmse:.2f}")
print(f"RF-GS, Train  R2 {r2_tr:.3f}, MAE {mae_tr:.2f}, RMSE {rmse_tr:.2f}")
print(f"RF-GS, Test   R2 {r2_te:.3f}, MAE {mae_te:.2f}, RMSE {rmse_te:.2f}")


RF-GS, CV R2 moyen 0.263, CV MAE moyen 18.69, CV RMSE moyen 24.33
RF-GS, Train  R2 0.729, MAE 11.29, RMSE 14.84
RF-GS, Test   R2 0.334, MAE 19.18, RMSE 25.21


utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle

In [47]:
# Importances de variables de la Random Forest
importances = best_rf.feature_importances_
feat_names = Xtr.columns
imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)

print("Top 20 des features les plus importantes")
print(imp.head(20).to_string())

Top 20 des features les plus importantes
YearBuilt                                            0.082866
Latitude                                             0.070802
share_primary_gfa                                    0.069956
LargestPropertyUseTypeGFA                            0.068147
PropertyGFABuilding(s)                               0.065501
Longitude                                            0.060355
NumberofFloors                                       0.037416
LargestPropertyUseType_Non-Refrigerated Warehouse    0.034515
PrimaryPropertyType_Other                            0.033941
PropertyGFAParking                                   0.033756
has_gas                                              0.033657
share_second_gfa                                     0.032419
SecondLargestPropertyUseTypeGFA                      0.027064
parking_share                                        0.021896
BuildingType_SPS-District K-12                       0.021032
PrimaryPropertyType_Warehouse